In [2]:
import sys
import numpy as np
import math
import pandas as pd
import random

from scipy import stats

%matplotlib inline

import matplotlib.pyplot as plt 
import matplotlib
matplotlib.rc('font',family='serif')
matplotlib.rc('font',weight='bold')
matplotlib.rc('font',size=16)
matplotlib.rcParams['axes.labelweight'] = 'bold'

from recommender_functions import *

In [10]:
# Get the ratings by user ID: starts at user index 1 
r = pd.read_csv('goodbooks-10k/ratings.csv')
df_ratings = pd.DataFrame(r)

cut_u, cut_b = 175, 1
df_ratings['user_counts'] = df_ratings.groupby(['user_id'])['book_id'].transform('count')
df_ratings['book_counts'] = df_ratings.groupby(['book_id'])['user_id'].transform('count')
df_ratings_cut = df_ratings.query('user_counts > %d'%(cut_u)) #df_ratings.copy() # 

book_ids = df_ratings_cut.book_id.values

In [11]:
print(df_ratings_cut.shape,df_ratings_cut.shape)

(98172, 5) (98172, 5)


In [12]:
# Get the ratings by book 
r = pd.read_csv('goodbooks-10k/books.csv')
df_books = pd.DataFrame(r)
df_books.head()

# Get only books that are left after our cut on #ratings/user and /book above
book_ids = df_ratings_cut.book_id.values
df_remaining_books = df_books[df_books['book_id'].isin(book_ids)]
df_remaining_books.head()

df_remaining_books.loc[:,'genre'] = 'default'
df_remaining_books.loc[:,'tag_id'] = -1
print(df_remaining_books.shape)
# Get goodreads_ids to siphon down tags in next section
goodreads_book_ids = df_remaining_books['goodreads_book_id'].values


(7337, 25)


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [28]:
#print("var countries=[",end='')
for i in range(df_remaining_books.shape[0]):
    title = df_remaining_books.title.values[i]

    if "'" in title:
        print (i, title)
#    if i == 222:
    #print ('"%s",'%df_remaining_books.title.values[i],end='')
#print("];",end='')


1 Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
37 The Time Traveler's Wife
53 The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)
58 Charlotte's Web
63 My Sister's Keeper
69 Ender's Game (Ender's Saga, #1)
74 Bridget Jones's Diary (Bridget Jones, #1)
110 The Memory Keeper's Daughter
118 The Handmaid's Tale
128 One Flew Over the Cuckoo's Nest
139 The Girl Who Kicked the Hornet's Nest (Millennium, #3)
178 Angela's Ashes (Frank McCourt, #1)
198 Marley and Me: Life and Love With the World's Worst Dog
213 The Titan's Curse (Percy Jackson and the Olympians, #3)
228 Alice's Adventures in Wonderland & Through the Looking-Glass
230 Sarah's Key
234 The Husband's Secret
247 A Midsummer Night's Dream
252 The Cuckoo's Calling (Cormoran Strike, #1)
255 Three Cups of Tea: One Man's Mission to Promote Peace ... One School at a Time
296 Cat's Cradle
298 Heaven is for Real: A Little Boy's Astounding Story of His Trip to Heaven and Back
305 She's Come Undone
306 The Wi

In [26]:
#df_remaining_books.head()

In [27]:
goodreads_to_book_id_map = {df_remaining_books['goodreads_book_id'].values[i] :df_remaining_books['book_id'].values[i] \
                            for i in range(df_remaining_books.shape[0])  }

In [28]:
# Tag_id, Tag_name
r = pd.read_csv('goodbooks-10k/ahack_tags_3.csv')
df_tags = pd.DataFrame(r)

# Get all 
tag_id = df_tags.tag_id.values

In [29]:
# goodreads_book_id,Tag_id,#ofinstances 
r = pd.read_csv('goodbooks-10k/book_tags.csv')
df_book_to_tags = pd.DataFrame(r)

In [30]:
# a -> narrow books down to the tags I myself narrowed down (got rid of numbers,dashes) 
a = df_book_to_tags[df_book_to_tags['tag_id'].isin(tag_id)]
#print(a.head())
# b -> narrow further by requiring goodreads ids to match the remaining books.csv above
b = a[a['goodreads_book_id'].isin(goodreads_book_ids)]
# c -> add count column so that can select most popular genre
c = b[b.groupby(['goodreads_book_id'])['count'].transform(max) == b['count']]

print(df_book_to_tags.shape,a.shape,b.shape,c.shape)
c.head()
#print(a.shape,b.shape,c.shape)
print(b.head())
print(c.head())
#plt.hist(c['count'].values)
#plt.yscale('log')

(999912, 3) (70269, 3) (70269, 3) (10067, 3)
    goodreads_book_id  tag_id  count
1                   1   11305  37174
16                  1    1691   1742
23                  1    9221    987
32                  1   17124    748
52                  1   20939    465
     goodreads_book_id  tag_id  count
1                    1   11305  37174
102                  2   11305   3441
202                  3   11305  47478
301                  5   11305  39330
400                  6   11305  38378


In [31]:
goodreads_id_to_tags_map = { c['goodreads_book_id'].values[i] : c['tag_id'].values[i]\
                    for i in range(c.shape[0])}

In [32]:
print(goodreads_id_to_tags_map[4948],len(goodreads_id_to_tags_map))
#print(tags_to_genre_map[goodreads_id_to_tags_map[4948]])

17124 9999


In [33]:
#plt.hist(goodreads_id_to_tags_map.values())

In [34]:
goodreads_ids_to_tag_ids = c.groupby('tag_id')['goodreads_book_id'].apply(list)
#print(goodreads_ids_to_tag_ids.shape)

df_tags_2 = df_tags[df_tags['tag_id'].isin(c.tag_id.unique())]
print(df_tags.shape,df_tags_2.shape)
df_tags_2.head()

tags_to_genre_map = { df_tags_2['tag_id'].values[i] : df_tags_2['tag_name'].values[i]\
                    for i in range(df_tags_2.shape[0])}

(58, 2) (57, 2)


In [35]:
print(goodreads_id_to_tags_map[1])

11305


In [36]:
print('C shape: ', c.shape[0])
j = 0
for i in goodreads_id_to_tags_map.keys(): #c.shape[0]):

    gr_id = i
    
    book_q = df_remaining_books.query('goodreads_book_id == %d'%int(gr_id))
    title = book_q['title'].values[0]
    tag_id = goodreads_id_to_tags_map[gr_id]      
    genre = tags_to_genre_map[tag_id]  
    df_remaining_books['genre'] = np.where(df_remaining_books.goodreads_book_id == int(gr_id),\
                                           genre, df_remaining_books.genre) 
    df_remaining_books['tag_id'] = np.where(df_remaining_books.goodreads_book_id == int(gr_id),\
                                           tag_id, df_remaining_books.tag_id) 

    
    if j % 500 == 0:
        print('here we are on i',j,title,genre)

    j += 1

C shape:  10067
here we are on i 0 Harry Potter and the Half-Blood Prince (Harry Potter, #6) fantasy
here we are on i 500 V for Vendetta dystopian
here we are on i 1000 Excession (Culture, #5) default
here we are on i 1500 Where the Wild Things Are fantasy
here we are on i 2000 Ina May's Guide to Childbirth parenting
here we are on i 2500 The Divan poetry
here we are on i 3000 Kiss an Angel romance
here we are on i 3500 The Bhagavad Gita religion
here we are on i 4000 The Vile Village (A Series of Unfortunate Events, #7) fantasy
here we are on i 4500 Just a Mess (Little Critter) kids
here we are on i 5000 Ariel poetry
here we are on i 5500 Nothing's Fair in Fifth Grade kids
here we are on i 6000 Snakehead (Alex Rider, #7) adventure
here we are on i 6500 The Winner Stands Alone thriller
here we are on i 7000 Bayou Moon (The Edge, #2) fantasy
here we are on i 7500 The Darkest Secret (Lords of the Underworld #7) romance
here we are on i 8000 Mile 81 horror
here we are on i 8500 Breathe (C

In [37]:
# print('C shape: ', c.shape[0])
# j = 0
# for i in goodreads_id_to_tags_map.keys(): #c.shape[0]):
#     #print(i,goodreads_id_to_tags_map[i])
#     #gr_id = c['goodreads_book_id'].values[i]
#     gr_id = i
    
#     book_q = df_remaining_books.query('goodreads_book_id == %d'%int(gr_id))
#     title = book_q['title'].values[0]
            
#     df_remaining_books['genre'] = np.where(df.good_reads_book_id == int(gr_id), 1036.8 - df.reco_vtx_z, df.reco_vtx_z) 

    
#     if book_q.shape[0] > 0:
#         if j % 500 == 0:
#             print('here we are on i',j,book_q.shape[0])
#         tag_id = goodreads_id_to_tags_map[gr_id]
#         genre = tags_to_genre_map[tag_id]
#         book_q['tag_id'].values[0] = int(tag_id)
#         book_q['genre'].values[0] = genre
#         if title == 'The Very Hungry Caterpillar Board Book':
#             print('ID: ',gr_id,tag_id,genre,book_q.goodreads_book_id.values,book_q.genre.values)
#     j += 1

In [38]:
#plt.hist(df_remaining_books.tag_id.values)

In [39]:
df_remaining_books.to_csv("remainin_books.csv", sep=',')#, encoding='utf-8')
df_remaining_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,genre,tag_id
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,dystopian,10064
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,fantasy,11305
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,fantasy,11305
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,classic,7404
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,classic,7404


In [19]:
countries = ["Afghanistan","Albania","Algeria","Andorra","Angola","Anguilla","Antigua & Barbuda","Argentina","Armenia","Aruba","Australia","Austria","Azerbaijan","Bahamas","Bahrain","Bangladesh","Barbados","Belarus","Belgium","Belize","Benin","Bermuda","Bhutan","Bolivia","Bosnia & Herzegovina","Botswana","Brazil","British Virgin Islands","Brunei","Bulgaria","Burkina Faso","Burundi","Cambodia","Cameroon","Canada","Cape Verde","Cayman Islands","Central Arfrican Republic","Chad","Chile","China","Colombia","Congo","Cook Islands","Costa Rica","Cote D Ivoire","Croatia","Cuba","Curacao","Cyprus","Czech Republic","Denmark","Djibouti","Dominica","Dominican Republic","Ecuador","Egypt","El Salvador","Equatorial Guinea","Eritrea","Estonia","Ethiopia","Falkland Islands","Faroe Islands","Fiji","Finland","France","French Polynesia","French West Indies","Gabon","Gambia","Georgia","Germany","Ghana","Gibraltar","Greece","Greenland","Grenada","Guam","Guatemala","Guernsey","Guinea","Guinea Bissau","Guyana","Haiti","Honduras","Hong Kong","Hungary","Iceland","India","Indonesia","Iran","Iraq","Ireland","Isle of Man","Israel","Italy","Jamaica","Japan","Jersey","Jordan","Kazakhstan","Kenya","Kiribati","Kosovo","Kuwait","Kyrgyzstan","Laos","Latvia","Lebanon","Lesotho","Liberia","Libya","Liechtenstein","Lithuania","Luxembourg","Macau","Macedonia","Madagascar","Malawi","Malaysia","Maldives","Mali","Malta","Marshall Islands","Mauritania","Mauritius","Mexico","Micronesia","Moldova","Monaco","Mongolia","Montenegro","Montserrat","Morocco","Mozambique","Myanmar","Namibia","Nauro","Nepal","Netherlands","Netherlands Antilles","New Caledonia","New Zealand","Nicaragua","Niger","Nigeria","North Korea","Norway","Oman","Pakistan","Palau","Palestine","Panama","Papua New Guinea","Paraguay","Peru","Philippines","Poland","Portugal","Puerto Rico","Qatar","Reunion","Romania","Russia","Rwanda","Saint Pierre & Miquelon","Samoa","San Marino","Sao Tome and Principe","Saudi Arabia","Senegal","Serbia","Seychelles","Sierra Leone","Singapore","Slovakia","Slovenia","Solomon Islands","Somalia","South Africa","South Korea","South Sudan","Spain","Sri Lanka","St Kitts & Nevis","St Lucia","St Vincent","Sudan","Suriname","Swaziland","Sweden","Switzerland","Syria","Taiwan","Tajikistan","Tanzania","Thailand","Timor L'Este","Togo","Tonga","Trinidad & Tobago","Tunisia","Turkey","Turkmenistan","Turks & Caicos","Tuvalu","Uganda","Ukraine","United Arab Emirates","United Kingdom","United States of America","Uruguay","Uzbekistan","Vanuatu","Vatican City","Venezuela","Vietnam","Virgin Islands (US)","Yemen","Zambia","Zimbabwe"]

In [20]:
print(len(countries))

222
